# 9. 🦄 가랏, 몬스터볼! 전설의 포켓몬 찾아 삼만리

## 9-1. 들어가며

### 학습 전제
---
- numpy, pandas를 활용해서 표 데이터를 다뤄본 적이 있고, 이에 대한 기본적인 활용을 할 수 있습니다.
- pandas의 문법과 다양한 메서드를 활용해 본 적이 있고, 코드를 보면 어느 정도 이해할 수 있습니다.
- matplotlib을 활용해서 데이터 시각화를 해본 적이 있고, 코드를 보면 어느 정도 이해할 수 있습니다.
- 데이터셋을 train/test 데이터셋으로 나누어서 모델을 학습 및 검증해본 경험이 있습니다.

### 학습 목표
---
- 다양한 피처가 있는 데이터셋을 밑바닥부터 샅샅이 뜯어보고, 전설의 포켓몬을 분류하기 위한 피처에는 무엇이 있는지 생각해 봅시다.
- 모델 학습을 시작하기 전 모든 컬럼에 대해 그래프 시각화, 피벗 테이블 등을 활용하며 다양한 방법으로 충분한 EDA를 진행합니다.
- 모델 학습에 넣기 위해서 전처리가 필요한 범주형/문자열 데이터에 대한 전처리를 원-핫 인코딩 등으로 적절하게 진행합니다.
- 전체 데이터셋을 train/test 데이터셋으로 나누고, 적절한 분류 모델(Decision Tree)을 선택해 학습시키며 베이스라인과 비교합니다.

### 목차
---
1. 포켓몬, 그 데이터는 어디서 구할까
    (1) 안녕, 포켓몬과 인사해!
    (2) 포켓몬, 그 데이터는 어디서 구할까
    (3) 포켓몬 데이터 불러오기
2. 전설의 포켓몬? 먼저 샅샅이 살펴보자!
    (1) 결측치와 전체 칼럼
    (2) ID와 이름
    (3) 포켓몬의 속성
    (4) 모든 스탯의 총합
    (5) 세부 스탯
    (6) 세대
3. 전설의 포켓몬과 일반 포켓몬, 그 차이는?
    (1) 전설의 포켓몬의 Total값
    (2) 전설의 포켓몬의 이름
4. 모델에 넣기 위해! 데이터 전처리하기
    (1) 이름의 길이가 10 이상인가?
    (2) 이름에 자주 쓰이는 토큰 추출
    (3) Type 1 & 2! 범주형 데이터 전처리하기
5. 가랏, 몬스터볼!
    (1) 가장 기본 데이터로 만드는 베이스라인
    (2) 의사 결정 트리 모델 학습시키기
    (3) 피처 엔지니어링 데이터로 학습시키면 얼마나 차이가 날까?
    (4) 의사 결정 트리 모델 다시 학습시키기

### 9-2. 안녕, 포켓몬과 인사해!

**전설의 포켓몬 구별하기**

탐색적 데이터 분석(Exploratory Data Analysis, 이하 EDA)

### 9-3. 포켓몬, 그 데이터는 어디서 구할까?

- 캐글 포켓몬 스탯 (https://www.kaggle.com/datasets/abcsds/pokemon)


## 9-4. 포켓몬 데이터 불러오기

### 라이브러리 가져오기
---

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
import os
csv_path = os.getenv("HOME") +"/aiffel/workplace/240527-pokemon-eda/data/Pokemon.csv"
original_data = pd.read_csv(csv_path)

In [4]:
pokemon = original_data.copy()
print(pokemon.shape)
pokemon.head()

(800, 13)


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [6]:
# 전설의 포켓몬 데이터셋
#
legendary = pokemon[pokemon["Legendary"] == True].reset_index(drop=True)
print(legendary.shape)
legendary.head()

(65, 13)


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,144,Articuno,Ice,Flying,580,90,85,100,95,125,85,1,True
1,145,Zapdos,Electric,Flying,580,90,90,85,125,90,100,1,True
2,146,Moltres,Fire,Flying,580,90,100,90,125,85,90,1,True
3,150,Mewtwo,Psychic,NaN,680,106,110,90,154,90,130,1,True
4,150,MewtwoMega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True


In [8]:
# Q. 일반 포켓몬의 데이터셋도 만들어봅시다.
ordinary = pokemon[pokemon["Legendary"] == False].reset_index(drop=True)
print(ordinary.shape)
ordinary.head()

(735, 13)


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
